<a href="https://colab.research.google.com/github/RayNCode/code_collab/blob/main/ML_Merged_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a first attempt to deal with missing values.

In [53]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [54]:
df = pd.read_csv("/content/learn_dataset.csv")

In [55]:
dept_code = pd.read_csv("/content/city_adm.csv")
df_1 = df.merge(dept_code, on='insee_code')

In [56]:
emp_contract = pd.read_csv("/content/learn_dataset_Emp_contract.csv")
df_2 = df_1.merge(emp_contract, on='Id', how='left')

In [57]:
reg_code = pd.read_csv("/content/departments.csv")
df_3 = df_2.merge(reg_code, on='dep', how='left')

In [96]:
from sklearn.model_selection import train_test_split
import sklearn
sklearn.set_config(display="diagram")
X = df_3.drop(["target", "Id", "Nom du département", 'town_type', 'insee_code', "Nom de la commune"], axis="columns")
y = df_3['target'].copy()
y = np.where(y == 'B', 1, 0)
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
categorical_columns = X.select_dtypes(include=['object']).columns
# Remplacer les valeurs manquantes par "None" dans les colonnes catégorielles
X[categorical_columns] = X[categorical_columns].fillna("None")
# Afficher les premières lignes du DataFrame pour vérifier
X

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [99]:
X.columns

Index(['is_student', 'OCCUPATION_42', 'ACTIVITY_TYPE', 'AGE_2019', 'household',
       'Highest_degree', 'sex', 'Emp_contract', 'REG'],
      dtype='object')

In [100]:
preprocessing = make_column_transformer (
    (OneHotEncoder(handle_unknown='ignore'), ["is_student", "OCCUPATION_42", "ACTIVITY_TYPE", "household", "sex", "dep", "Emp_contract", "REG"]),
    (OrdinalEncoder(), ["Highest_degree"]),
    remainder='passthrough')

In [101]:
logreg = LogisticRegression(solver='lbfgs', max_iter=5000)
pipe_logreg = make_pipeline(preprocessing, logreg)
pipe_logreg

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['is_student',
                                                   'OCCUPATION_42',
                                                   'ACTIVITY_TYPE', 'household',
                                                   'sex', 'Emp_contract',
                                                   'REG']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Highest_degree'])])),
                ('logisticregression', LogisticRegression(max_iter=5000))])

In [102]:
cross_val_score(pipe_logreg, X_train, y_train, cv=3, scoring="accuracy").mean()

0.7729658945941104

In [103]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
pipe_forest = make_pipeline(preprocessing, forest_clf)

In [80]:
cross_val_score(pipe_forest, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1).mean()

0.8043206451068949

In [84]:
pipe_forest.fit(X_train, y_train)  # Assurez-vous de fournir vos données d'entraînement X_train et y_train

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['is_student',
                                                   'OCCUPATION_42',
                                                   'ACTIVITY_TYPE', 'household',
                                                   'sex', 'dep', 'Emp_contract',
                                                   'REG']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Highest_degree'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [ ]:
# Accédez au modèle RandomForestClassifier à l'intérieur du pipeline
forest_model = pipe_forest.named_steps['randomforestclassifier']

# Obtenez les importances des caractéristiques du modèle
feature_importances = forest_model.feature_importances_

In [95]:
# Accédez à l'objet ColumnTransformer dans votre pipeline.
preprocessor = pipe_forest.named_steps['columntransformer']

# Utilisez la méthode get_feature_names_out() pour obtenir les noms des features.
feature_names = preprocessor.get_feature_names_out()

# Reliez ces noms aux importances des caractéristiques pour les interpréter.
feature_importances = pipe_forest.named_steps['randomforestclassifier'].feature_importances_

feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Trier le DataFrame par importance pour voir les caractéristiques les plus importantes.
feature_importances_df.sort_values(by='Importance', ascending=False)



,Feature,Importance
180,remainder__AGE_2019,0.155387
179,ordinalencoder__Highest_degree,0.085298
43,onehotencoder__ACTIVITY_TYPE_TYPE1|1,0.031697
165,onehotencoder__Emp_contract_None,0.021947
56,onehotencoder__household_TYPMR4-3,0.015353
...,...,...
4,onehotencoder__OCCUPATION_42_csp_1_3,0.000338
159,onehotencoder__Emp_contract_COND.1.4,0.000269
16,onehotencoder__OCCUPATION_42_csp_4_4,0.000127
164,onehotencoder__Emp_contract_COND.2.3,0.000091
